In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Load Packages
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns


In [3]:
data = pd.read_csv('/kaggle/input/t2d-abundance-microbiome/T2D_abundance.csv', delimiter = '\t', index_col = None)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/t2d-abundance-microbiome/T2D_abundance.csv'

In [ ]:
data.rename( columns={'Unnamed: 0':'Index'}, inplace=True )
data.head()

In [ ]:
data.set_index('Index')
data.head()

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum().head()

In [ ]:
features = data.columns

In [ ]:
for col in features:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [ ]:
data.isnull().sum().head()

In [ ]:
y = data['Class']

X = data.drop(['Index','Class'], axis=1)

In [ ]:
pca = PCA()
df_pca = pca.fit_transform(X=X)

# Store as dataframe and print
df_pca = pd.DataFrame(df_pca)
print(df_pca.shape)  #> (3147, 784)
df_pca.round(2).head()

In [ ]:
pca.explained_variance_ratio_.round(8)

In [ ]:
variance_exp_cumsum = pca.explained_variance_ratio_.cumsum().round(2)
fig, axes = plt.subplots(1,1,figsize=(16,7), dpi=100)
plt.plot(variance_exp_cumsum, color='red')
plt.xlabel('# of PCs', fontsize=16)
plt.ylabel('Variance', fontsize=16)
plt.show()

In [ ]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
dim = np.argmax(cumsum >= 0.95) + 1
print('The number of dimensions required to preserve 95% of variance is',dim)

Code below is refered from: https://www.machinelearningplus.com/machine-learning/principal-components-analysis-pca-better-explained/

In [ ]:
from scipy.spatial import ConvexHull

def encircle(x,y, ax=None, **kw):
    if not ax: ax=plt.gca()
    p = np.c_[x,y]
    hull = ConvexHull(p)
    poly = plt.Polygon(p[hull.vertices,:], **kw)
    ax.add_patch(poly)

In [ ]:
# Scatterplot against PC1 and PC2
fig, ax = plt.subplots(1,1, figsize=(16,12))

# Row masks for each category
rows_0 = y=='n';
rows_1 = y=='t2d'; 

# Plot
ax.scatter(df_pca.loc[rows_0.tolist(), 1], df_pca.loc[rows_0.tolist(), 2], c='blue', edgecolor='k', s=120, label='No')
ax.scatter(df_pca.loc[rows_1.tolist(), 1], df_pca.loc[rows_1.tolist(), 2], c='red', edgecolor='k', s=120, label='Yes')

# # Encircle the boundaries
# encircle(df_pca.loc[rows_0.tolist(), 1], df_pca.loc[rows_0.tolist(), 2], ec="blue", fc="none", linewidth=2.5)
# encircle(df_pca.loc[rows_1.tolist(), 1], df_pca.loc[rows_1.tolist(), 2], ec="firebrick", fc="none", linewidth=2.5)

# # Shading
# encircle(df_pca.loc[rows_1.tolist(), 1], df_pca.loc[rows_1.tolist(), 2], ec="k", fc="firebrick", alpha=0.05)
# encircle(df_pca.loc[rows_0.tolist(), 1], df_pca.loc[rows_0.tolist(), 2], ec="k", fc="blue", alpha=0.05)

# Labels
ax.set_title("Microbiome Data for 1 & 2 : Scatterplot of First Two PCA directions", fontsize=22)
ax.set_xlabel("1st Principal Component", fontsize=22)
ax.set_ylabel("2nd Principal Component", fontsize=22)
ax.legend(loc='upper right', title='Diabetes', fontsize=16)
plt.show();

In [ ]:
data['TypeClass'] = data['Class'].rank(method='dense').astype(int)

In [ ]:
np.random.seed(4)
rndperm = np.random.permutation(data.shape[0])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
X = data.drop(['Index','Class'], axis = 1)
features = X.columns

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(data[features].values)
data['pca-one'] = pca_result[:,0]
data['pca-two'] = pca_result[:,1] 
data['pca-three'] = pca_result[:,2]


In [ ]:
# 3-D Scatterplot against PC1, PC2 and PC3
ax = plt.figure(figsize=(16,12)).gca(projection='3d')

# Row masks for each category
rows_0 = y=='n';
rows_1 = y=='t2d'; 

ax.scatter(
    xs=data.loc[rndperm,:]["pca-one"], 
    ys=data.loc[rndperm,:]["pca-two"], 
    zs=data.loc[rndperm,:]["pca-three"], 
    c=data.loc[rndperm,:]["TypeClass"], 
    cmap='tab10'
)

ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show();

# t-SNE implementation:

In [ ]:
data_subset = data[features].values
pca = PCA(n_components=3)
pca_result = pca.fit_transform(data_subset)

data['pca-one'] = pca_result[:,0]
data['pca-two'] = pca_result[:,1] 
data['pca-three'] = pca_result[:,2]

print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))


In [ ]:
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_subset)
# print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))
tsne_results = pd.DataFrame(tsne_results)
tsne_results

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,12))

rows_0 = y=='n';
rows_1 = y=='t2d'; 

# Plot
ax.scatter(tsne_results.loc[rows_0.tolist(), 1], tsne_results.loc[rows_0.tolist(), 2], c='blue', edgecolor='k', s=120, label='No')
ax.scatter(tsne_results.loc[rows_1.tolist(), 1], tsne_results.loc[rows_1.tolist(), 2], c='red', edgecolor='k', s=120, label='Yes')

# data['tsne-2d-one'] = tsne_results[:,0]
# data['tsne-2d-two'] = tsne_results[:,1]

ax.set_title("Microbiome Data for 1 & 2 : Scatterplot of First Two t-SNE directions", fontsize=22)
ax.set_xlabel("tsne-2d-one", fontsize=22)
ax.set_ylabel("tsne-2d-two", fontsize=22)
ax.legend(loc='upper right', title='Diabetes', fontsize=16)
plt.show();

In [ ]:
data['tsne-3d-one'] = tsne_results[:,0]
data['tsne-3d-two'] = tsne_results[:,1]
data['tsne-3d-three'] = tsne_results[:,2]


In [ ]:
ax = plt.figure(figsize=(16,12)).gca(projection='3d')

# Row masks for each category
rows_0 = y=='n';
rows_1 = y=='t2d'; 

ax.scatter(
    xs=data.loc[rndperm,:]["tsne-3d-one"], 
    ys=data.loc[rndperm,:]["tsne-3d-two"], 
    zs=data.loc[rndperm,:]["tsne-3d-three"], 
    c=data.loc[rndperm,:]["TypeClass"], 
    cmap='tab10'
)

ax.set_xlabel('tsne-one')
ax.set_ylabel('tsne-two')
ax.set_zlabel('tsne-three')
plt.show();

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math

deltas = []
for N in range(2,50,1):
    P = [np.random.randint(-100, 100, N) for _ in range(500)]
    Q = np.random.randint(-100,100,N)
    diffs = [np.linalg.norm(p-Q) for p in P]
    mxd = max(diffs)
    mnd = min(diffs)
    delta = math.log10(mxd-mnd)/mnd
    deltas.append( delta )

plt.plot(range(2,50,1),deltas)
plt.show()